In [16]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import os
import time
from pathlib import Path

In [17]:
# load mnist
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# normalize
train_images = train_images / 255.0

train_labels = tf.one_hot(train_labels, 10).numpy()
test_labels = tf.one_hot(test_labels, 10).numpy()

# split train and validation
split_index = int(len(train_images) * 0.8)
val_images = train_images[split_index:]
val_labels = train_labels[split_index:]
train_images = train_images[:split_index]
train_labels = train_labels[:split_index]

train_images = train_images[..., np.newaxis]
val_images = val_images[..., np.newaxis]
test_images = test_images[..., np.newaxis]


In [18]:
def get_log_path():
    root_logdir = os.path.join(os.curdir, "my_logs")
    run_id = time.strftime("%Y_%m_%d-%H_%M")
    return os.path.join(root_logdir, run_id)

def build_model(learning_rate=1e-3, l2_lambda=0.01, dropout_rate=0.3):
    model = keras.models.Sequential([
        keras.layers.Conv2D(32, (3, 3), kernel_regularizer=keras.regularizers.l2(l2_lambda), padding="same"),
        keras.layers.Activation("relu"),
        keras.layers.Conv2D(64, (3, 3), kernel_regularizer=keras.regularizers.l2(l2_lambda), padding="same"),
        keras.layers.Activation("relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(dropout_rate/2),
        keras.layers.Dense(128, kernel_regularizer=keras.regularizers.l2(l2_lambda)),
        keras.layers.Activation("relu"),
        keras.layers.Dropout(dropout_rate),
        keras.layers.Dense(10, activation="softmax", kernel_regularizer=keras.regularizers.l2(l2_lambda))
    ])

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

    return model

In [19]:
epochs = 100
run_log_dir = get_log_path()
learning_rate = 1e-3
l2_reg = 0.00
batch_size = 16
early_stopping_patience = 5
lr_scheduler_factor = 0.5
lr_scheduler_patience = 3
batch_size = 64

model_name = "c32_c64_pooling_drop0.15_128_drop0.3"

early_stopping_cb = keras.callbacks.EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=lr_scheduler_factor, patience=lr_scheduler_patience)

run_log_dir = f"{model_name}_{run_log_dir}"

tensorboard_cb = keras.callbacks.TensorBoard(run_log_dir)
model = build_model(learning_rate=learning_rate, l2_lambda=l2_reg)
model.fit(train_images, train_labels, epochs=epochs, validation_data=(val_images, val_labels), 
            callbacks=[keras.callbacks.TensorBoard(run_log_dir), early_stopping_cb, lr_scheduler], batch_size=batch_size)
model.save(f"{run_log_dir}/{model_name}.keras")
loss, acc = model.evaluate(test_images, test_labels)
print(f"{model_name} Accuracy: {acc} Loss: {loss}")
with open(f"{run_log_dir}/results.txt", "a") as f:
    f.write(f"{model_name} Accuracy: {acc} Loss: {loss}\n")

Epoch 1/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 196s 257ms/step - accuracy: 0.8964 - loss: 0.3313 - val_accuracy: 0.9861 - val_loss: 0.0481 - learning_rate: 0.0010
Epoch 2/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 265s 354ms/step - accuracy: 0.9821 - loss: 0.0570 - val_accuracy: 0.9885 - val_loss: 0.0422 - learning_rate: 0.0010
Epoch 3/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 252s 336ms/step - accuracy: 0.9871 - loss: 0.0398 - val_accuracy: 0.9893 - val_loss: 0.0404 - learning_rate: 0.0010
Epoch 4/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 240s 320ms/step - accuracy: 0.9895 - loss: 0.0302 - val_accuracy: 0.9897 - val_loss: 0.0386 - learning_rate: 0.0010
Epoch 5/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 257s 342ms/step - accuracy: 0.9931 - loss: 0.0233 - val_accuracy: 0.9877 - val_loss: 0.0486 - learning_rate: 0.0010
Epoch 6/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 244s 326ms/step - accuracy: 0.9942 - loss: 0.0185 - val_accuracy: 0.9924 - val_loss: 0.0333 - learning_rate: 0.0010
Epoch 7/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 237s 316ms/step -